In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-f6bbbd96e95b>:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
lines = pd.read_csv('/content/drive/MyDrive/Sem_8-G21/Datasets/gu-en.tsv',sep='\t',names=['english_sentence', 'gujarati_sentence'])
print(len(lines))
lines = lines[:40000]
lines.tail(3)

49844


,english_sentence,gujarati_sentence
39997,"The urban component has already been approved on 25th June, 2015 and is under implementation.","યોજનાના શહેરી હિસ્સાને 25મી જુન, 2015ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે."
39998,"Under the existing Rural Housing Scheme { Indira Awaas Yojana (IAY)} implemented by Ministry of Rural Development, financial assistance of Rs.70,000/- in plain areas and Rs.75,000/- in hilly/difficult areas including IAP districts, is provided to the rural BPL households for construction of a dwelling unit.","ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના {ઈન્દિરા આવાસ યોજના (IAY)} અંતર્ગત મેદાની, સામાન્ય વિસ્તારોમાં રૂ. 70,000 અને પર્વતીય/દુર્ગમ વિસ્તારો, આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ. 75,000 નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને (ગરીબીની રેખા હેઠળ જીવતા – BPL પરિવારોને) મકાન બાંધકામ માટે પુરી પડાય છે."
39999,"Since inception of the scheme, 351 lakh houses have been constructed, incurring a total expenditure of Rs 1,05,815.80 crore.","આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં 351 લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ. 1,05,815.80 કરોડનો ખર્ચ થયો છે."


In [ ]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: str(x))
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: str(x))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: x.lower())
lines.tail(3)

,english_sentence,gujarati_sentence
39997,"the urban component has already been approved on 25th june, 2015 and is under implementation.","યોજનાના શહેરી હિસ્સાને 25મી જુન, 2015ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે."
39998,"under the existing rural housing scheme { indira awaas yojana (iay)} implemented by ministry of rural development, financial assistance of rs.70,000/- in plain areas and rs.75,000/- in hilly/difficult areas including iap districts, is provided to the rural bpl households for construction of a dwelling unit.","ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના {ઈન્દિરા આવાસ યોજના (iay)} અંતર્ગત મેદાની, સામાન્ય વિસ્તારોમાં રૂ. 70,000 અને પર્વતીય/દુર્ગમ વિસ્તારો, આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ. 75,000 નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને (ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને) મકાન બાંધકામ માટે પુરી પડાય છે."
39999,"since inception of the scheme, 351 lakh houses have been constructed, incurring a total expenditure of rs 1,05,815.80 crore.","આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં 351 લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ. 1,05,815.80 કરોડનો ખર્ચ થયો છે."


In [ ]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: re.sub("'", '', x))
lines.tail(3)

,english_sentence,gujarati_sentence
39997,"the urban component has already been approved on 25th june, 2015 and is under implementation.","યોજનાના શહેરી હિસ્સાને 25મી જુન, 2015ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે."
39998,"under the existing rural housing scheme { indira awaas yojana (iay)} implemented by ministry of rural development, financial assistance of rs.70,000/- in plain areas and rs.75,000/- in hilly/difficult areas including iap districts, is provided to the rural bpl households for construction of a dwelling unit.","ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના {ઈન્દિરા આવાસ યોજના (iay)} અંતર્ગત મેદાની, સામાન્ય વિસ્તારોમાં રૂ. 70,000 અને પર્વતીય/દુર્ગમ વિસ્તારો, આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ. 75,000 નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને (ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને) મકાન બાંધકામ માટે પુરી પડાય છે."
39999,"since inception of the scheme, 351 lakh houses have been constructed, incurring a total expenditure of rs 1,05,815.80 crore.","આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં 351 લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ. 1,05,815.80 કરોડનો ખર્ચ થયો છે."


In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tail(3)

,english_sentence,gujarati_sentence
39997,the urban component has already been approved on 25th june 2015 and is under implementation,યોજનાના શહેરી હિસ્સાને 25મી જુન 2015ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે
39998,under the existing rural housing scheme indira awaas yojana iay implemented by ministry of rural development financial assistance of rs70000 in plain areas and rs75000 in hillydifficult areas including iap districts is provided to the rural bpl households for construction of a dwelling unit,ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના ઈન્દિરા આવાસ યોજના iay અંતર્ગત મેદાની સામાન્ય વિસ્તારોમાં રૂ 70000 અને પર્વતીયદુર્ગમ વિસ્તારો આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ 75000 નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને મકાન બાંધકામ માટે પુરી પડાય છે
39999,since inception of the scheme 351 lakh houses have been constructed incurring a total expenditure of rs 10581580 crore,આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં 351 લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ 10581580 કરોડનો ખર્ચ થયો છે


In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: x.translate(remove_digits))
lines['gujarati_sentence'] = lines['gujarati_sentence'].apply(lambda x: re.sub("[૦૧૨૩૪૫૬૭૮૯]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['gujarati_sentence']=lines['gujarati_sentence'].apply(lambda x: re.sub(" +", " ", x))

lines.tail(3)

,english_sentence,gujarati_sentence
39997,the urban component has already been approved on th june and is under implementation,યોજનાના શહેરી હિસ્સાને મી જુન ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે
39998,under the existing rural housing scheme indira awaas yojana iay implemented by ministry of rural development financial assistance of rs in plain areas and rs in hillydifficult areas including iap districts is provided to the rural bpl households for construction of a dwelling unit,ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના ઈન્દિરા આવાસ યોજના iay અંતર્ગત મેદાની સામાન્ય વિસ્તારોમાં રૂ અને પર્વતીયદુર્ગમ વિસ્તારો આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને મકાન બાંધકામ માટે પુરી પડાય છે
39999,since inception of the scheme lakh houses have been constructed incurring a total expenditure of rs crore,આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ કરોડનો ખર્ચ થયો છે


In [ ]:
# Add start and end tokens to target sequences
lines['gujarati_sentence'] = lines['gujarati_sentence'].apply(lambda x : 'START_ '+ x + ' _END')
lines.tail(3)

,english_sentence,gujarati_sentence
39997,the urban component has already been approved on th june and is under implementation,START_ યોજનાના શહેરી હિસ્સાને મી જુન ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે _END
39998,under the existing rural housing scheme indira awaas yojana iay implemented by ministry of rural development financial assistance of rs in plain areas and rs in hillydifficult areas including iap districts is provided to the rural bpl households for construction of a dwelling unit,START_ ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના ઈન્દિરા આવાસ યોજના iay અંતર્ગત મેદાની સામાન્ય વિસ્તારોમાં રૂ અને પર્વતીયદુર્ગમ વિસ્તારો આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને મકાન બાંધકામ માટે પુરી પડાય છે _END
39999,since inception of the scheme lakh houses have been constructed incurring a total expenditure of rs crore,START_ આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ કરોડનો ખર્ચ થયો છે _END


In [ ]:
### Get English and Gujarati Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_gujarati_words=set()
for guj in lines['gujarati_sentence']:
    for word in guj.split():
        if word not in all_gujarati_words:
            all_gujarati_words.add(word)

len(all_eng_words), len(all_gujarati_words)

(26554, 56366)

In [ ]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_guj_sentence']=lines['gujarati_sentence'].apply(lambda x:len(x.split(" ")))
lines.tail(3)

,english_sentence,gujarati_sentence,length_eng_sentence,length_guj_sentence
39997,the urban component has already been approved on th june and is under implementation,START_ યોજનાના શહેરી હિસ્સાને મી જુન ના રોજ મંજુરી અપાઈ ગઈ છે અને તેના અમલીકરણનો આરંભ પણ થઈ ગયો છે _END,14,21
39998,under the existing rural housing scheme indira awaas yojana iay implemented by ministry of rural development financial assistance of rs in plain areas and rs in hillydifficult areas including iap districts is provided to the rural bpl households for construction of a dwelling unit,START_ ગ્રામિણ વિકાસ મંત્રાલય હસ્તક અમલીકરણ હેઠળની હયાત ગ્રામિણ આવાસ યોજના ઈન્દિરા આવાસ યોજના iay અંતર્ગત મેદાની સામાન્ય વિસ્તારોમાં રૂ અને પર્વતીયદુર્ગમ વિસ્તારો આઈએપી જિલ્લાઓ સહિતના વિસ્તારોમાં રૂ નાણાંકિય સહાય ગ્રામિણ ગરીબોના પરિવારોને ગરીબીની રેખા હેઠળ જીવતા – bpl પરિવારોને મકાન બાંધકામ માટે પુરી પડાય છે _END,44,47
39999,since inception of the scheme lakh houses have been constructed incurring a total expenditure of rs crore,START_ આ યોજનાનો આરંભ થયો ત્યારથી આજદિન સુધીમાં લાખ મકાનોનું બાંધકામ થયું છે અને તેની પાછળ કુલ રૂ કરોડનો ખર્ચ થયો છે _END,17,23


In [ ]:
lines[lines['length_eng_sentence']>20].shape,lines[lines['length_guj_sentence']>20].shape

((13493, 4), (13898, 4))

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_guj_sentence']<=20]
print(lines.shape)
lines.tail(3)

(23627, 4)


,english_sentence,gujarati_sentence,length_eng_sentence,length_guj_sentence
39991,the intangible benefits from improvement in housing condition are gains in labour productivity and positive health benefits,START_ રહેણાંકની સુવિધામાં સુધારાના દેખિતા ના હોય તેવા લાભોમાં શ્રમિકોની ઉત્પાદકતામાં ફાયદા તથા આરોગ્યના સકારાત્મક લાભનો સમાવેશ થાય છે _END,17,20
39992,it positively influences human development parameters of nutrition sanitation maternal and child health,START_ પોષણ સફાઈસ્વચ્છતા માતા તેમજ શિશુઓના આરોગ્ય જેવા માનવ વિકાસના માપદંડો ઉપર પણ તેનો સકારાત્મક પ્રભાવ વર્તાય છે _END,13,19
39993,overall improvement in quality of life occurs along with improvements in the physical environment,START_ ભૌતિક અને શારીરિક માહોલમાં સુધારાની સાથોસાથ એકંદરે જીવનની ગુણવત્તામાં પણ સુધારો થાય છે _END,14,15


In [ ]:
max_length_src=max(lines['length_eng_sentence'])
max_length_tar=max(lines['length_guj_sentence'])
print("maximum length of Gujarati Sentence ",max_length_src)
print("maximum length of English Sentence ",max_length_tar)
lines.tail(3)

maximum length of Gujarati Sentence  20
maximum length of English Sentence  20


,english_sentence,gujarati_sentence,length_eng_sentence,length_guj_sentence
39991,the intangible benefits from improvement in housing condition are gains in labour productivity and positive health benefits,START_ રહેણાંકની સુવિધામાં સુધારાના દેખિતા ના હોય તેવા લાભોમાં શ્રમિકોની ઉત્પાદકતામાં ફાયદા તથા આરોગ્યના સકારાત્મક લાભનો સમાવેશ થાય છે _END,17,20
39992,it positively influences human development parameters of nutrition sanitation maternal and child health,START_ પોષણ સફાઈસ્વચ્છતા માતા તેમજ શિશુઓના આરોગ્ય જેવા માનવ વિકાસના માપદંડો ઉપર પણ તેનો સકારાત્મક પ્રભાવ વર્તાય છે _END,13,19
39993,overall improvement in quality of life occurs along with improvements in the physical environment,START_ ભૌતિક અને શારીરિક માહોલમાં સુધારાની સાથોસાથ એકંદરે જીવનની ગુણવત્તામાં પણ સુધારો થાય છે _END,14,15


In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_gujarati_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
num_encoder_tokens, num_decoder_tokens

(26554, 56366)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
print(list(input_token_index.items())[0:10],"\n", list(target_token_index.items())[0:10])

[('a', 1), ('aa', 2), ('aabhaar', 3), ('aab”', 4), ('aadarsh', 5), ('aadhaar', 6), ('aadhaarlinkage', 7), ('aadhar', 8), ('aaedumnaan', 9), ('aaguner', 10)] 
 [('START_', 1), ('_END', 2), ('a', 3), ('aai', 4), ('about', 5), ('acceleratorના', 6), ('accept', 7), ('accessories', 8), ('account', 9), ('accountancy', 10)]


In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
print(list(reverse_input_char_index.items())[0:10],"\n", list(reverse_target_char_index.items())[0:10])

[(1, 'a'), (2, 'aa'), (3, 'aabhaar'), (4, 'aab”'), (5, 'aadarsh'), (6, 'aadhaar'), (7, 'aadhaarlinkage'), (8, 'aadhar'), (9, 'aaedumnaan'), (10, 'aaguner')] 
 [(1, 'START_'), (2, '_END'), (3, 'a'), (4, 'aai'), (5, 'about'), (6, 'acceleratorના'), (7, 'accept'), (8, 'accessories'), (9, 'account'), (10, 'accountancy')]


In [ ]:
lines.tail(5)

,english_sentence,gujarati_sentence,length_eng_sentence,length_guj_sentence
39988,the impacts occur in two phases during construction and during occupancy,START_ તેની અસરો બે તબક્કે વર્તાય છેઃ બાંધકામ દરમિયાન અને પછી તેમાં રહેણાંક વખતે _END,11,15
39989,the positive spinoffs include social integration including enhanced social capital and sustainable communities,START_ તેના સકારાત્મક પરિણામોમાં અધિક સામાજિક મૂડી અને સુદીર્ઘ સમુદાયો સહિતના સામાજિક એકીકરણનો સમાવેશ થાય છે _END,13,17
39991,the intangible benefits from improvement in housing condition are gains in labour productivity and positive health benefits,START_ રહેણાંકની સુવિધામાં સુધારાના દેખિતા ના હોય તેવા લાભોમાં શ્રમિકોની ઉત્પાદકતામાં ફાયદા તથા આરોગ્યના સકારાત્મક લાભનો સમાવેશ થાય છે _END,17,20
39992,it positively influences human development parameters of nutrition sanitation maternal and child health,START_ પોષણ સફાઈસ્વચ્છતા માતા તેમજ શિશુઓના આરોગ્ય જેવા માનવ વિકાસના માપદંડો ઉપર પણ તેનો સકારાત્મક પ્રભાવ વર્તાય છે _END,13,19
39993,overall improvement in quality of life occurs along with improvements in the physical environment,START_ ભૌતિક અને શારીરિક માહોલમાં સુધારાની સાથોસાથ એકંદરે જીવનની ગુણવત્તામાં પણ સુધારો થાય છે _END,14,15


In [ ]:
from sklearn.model_selection import train_test_split
X, y = lines['english_sentence'], lines['gujarati_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
print(X_train[0:5],'\n\n',X_test[0:5],'\n\n',y_train[0:5],'\n\n',y_test[0:5])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

4543     the purpose of auroville is to realise human unity                        
35530    guests from india and abroad                                              
19519    i would particularly urge my young friends to join tomorrow’s interaction 
35288    the state governments have braced themselves up for rescue and relief work
10375    he said top indian cricketers use bats made in the state                  
Name: english_sentence, dtype: object 

 2611     was pleased to visit haifa and pay homage there in person in july” the prime minister said                           
33233    · meetings of the brics working groups competition                                                                   
33789    major impact including employment generation potential                                                               
16455    mr kamal thapa deputy prime minister and foreign minister of nepal called on prime minister shri narendra modi today 
8400     they emphasised the im

((18901,), (4726,), (18901,), (4726,))

In [ ]:
# Tokenize the input and target sequences
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(X_train)
# Save tokenizer as a JSON string
tokenizer_json = tokenizer_inputs.to_json()
with open('input_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)
input_sequences_train = tokenizer_inputs.texts_to_sequences(X_train)
input_sequences_test = tokenizer_inputs.texts_to_sequences(X_test)
max_len_input = max(len(x) for x in input_sequences_train)
print(input_sequences_train,"\n\n",input_sequences_test,"\n\n",max_len_input)


tokenizer_output = Tokenizer()
tokenizer_output.fit_on_texts(y_train)
# Save tokenizer as a JSON string
tokenizer_output_json = tokenizer_output.to_json()
with open('output_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_output_json)
output_sequences_train = tokenizer_output.texts_to_sequences(y_train)
output_sequences_test = tokenizer_output.texts_to_sequences(y_test)
max_len_output = max(len(x) for x in output_sequences_train)
print(output_sequences_train,"\n\n",output_sequences_test,"\n\n",max_len_output)

[[1, 946, 2, 1810, 7, 4, 2782, 315, 858], [1811, 31, 19, 3, 1427], [23, 55, 979, 1428, 47, 240, 217, 4, 665, 6220, 587], [1, 84, 607, 18, 8504, 836, 80, 8, 1898, 3, 1116, 85], [17, 28, 1087, 54, 5040, 312, 8505, 113, 5, 1, 84], [588, 105, 7, 979, 111, 11, 1486, 6221], [3027, 61, 413, 5, 428, 385, 1566, 6, 3383, 5, 3773], [1, 235, 41, 10, 351, 54, 394, 3384, 5, 49, 8506], [1, 14, 13, 730, 1, 395, 2, 1, 3385, 608, 2, 1, 3774, 805, 2, 3386, 3, 8507], [9, 4296, 243, 10, 1280, 29, 1, 3387, 2, 71, 284, 4, 166], [39, 2, 1189, 18, 227, 46, 194, 8, 1, 99, 27, 1567, 5, 1, 859, 367, 1, 14, 13, 28], [16, 120, 22, 1899, 11, 58, 750], [68, 1, 131, 3775, 2, 2264, 24, 6222, 5041, 212, 11, 71, 2265], [1281, 41, 88, 6, 2592, 33, 18, 179, 171, 90, 8, 63], [1, 4297, 2, 3776, 2, 54, 731, 3777, 164, 1088, 64, 8508, 3778], [1900, 980, 2, 102, 7, 6, 545, 8, 63, 17, 142], [17, 26, 915, 6, 2593, 5042, 4, 3028, 1, 1725, 70, 2, 34, 5043, 8509], [11, 19, 806, 270, 3, 11, 1332], [23, 100, 396, 4, 2121, 16, 33, 20, 

In [ ]:
X_train

4543     the purpose of auroville is to realise human unity                                                                
35530    guests from india and abroad                                                                                      
19519    i would particularly urge my young friends to join tomorrow’s interaction                                         
35288    the state governments have braced themselves up for rescue and relief work                                        
10375    he said top indian cricketers use bats made in the state                                                          
                                   ...                                                                                     
20467    the indiaisrael innovation bridge will act as a link between the startups of the two sides                        
36581    in our media many interesting events of the year gone by are broadcast again to reflect on the previous year      
9440    

In [ ]:
target_token_index['START_'], target_token_index["_END"]

(1, 2)

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.0
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
'''
This looks like the summary of a neural network model created using Keras in Python.

The model has two input layers (input_4 and input_5) that are both expecting inputs of 
variable length (None in the shape). The first input is likely the input sequence, 
and the second input is probably the target sequence.

There are two embedding layers (embedding_3 and embedding_4) that are used to convert 
the input sequences into dense vector representations. The embedding layers have 300 
output dimensions, meaning each input word will be represented as a vector of length 300.

There are two LSTM layers (lstm_3 and lstm_4) that use the input embeddings to learn 
patterns in the input sequences. The first LSTM layer only returns its last hidden 
state, while the second LSTM layer returns the full sequence of hidden states.

Finally, there is a dense layer (dense_1) with 55,452 units, which is likely used 
to output the predicted sequence. The output shape is also of variable length, 
with None as the first dimension, meaning the model can predict sequences of any length.

The model has a total of 42,735,252 parameters that are all trainable, meaning 
they will be updated during training to minimize the loss function.
'''

model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    7966200     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    16910100    ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=1,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

<ipython-input-165-91434ddd982b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


295/295 [==============================] - 2020s 7s/step - loss: 7.7790 - accuracy: 0.1045 - val_loss: 7.2655 - val_accuracy: 0.1267


In [ ]:
model.save('eng-to-gujarati.h5')

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

        
    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
print(input_seq)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4]) # t to remove START_ and -4 to remove _END
print('Predicted Hindi Translation:', decoded_sentence[:-4]) # -4 to remove _END

[[22657. 17986. 15733.  1743. 11457. 22940. 18457. 10178. 23796.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]]
1/1 [==============================] - 0s 32ms/step
Input English sentence: the purpose of auroville is to realise human unity
Actual Hindi Translation:  ઑરોવિલેનો ઉદ્દેશ્ય માનવીય એકતાનીપ્રતીતી કરાવવાનો હતો 
Predicted Hindi Translation:  આ અને અને અને અને અને પણ છે 
